In [1]:
import collections 
import functools


import pandas as pd


In [2]:
cols = [
    "Annotator", "Annotation date", "Name", "GBE ID", "Field", "BasketID", "TableID", 
    "FieldID", "QT_total_num", "BIN_total_num", "QT_index", "BIN_index", "coding_exclude", 
    "coding_QT", "coding_binary_case", "coding_binary_control", "Participants", "Stability", 
    "ValueType", "Units", "Strata", "Sexed", "Instances", "Array", "Coding", "Link", "Notes"
]


In [3]:
def my_read_table(tbl, cols):
    df = pd.read_table(tbl, index_col=None, dtype=object)
    for c in set(cols) - set(df.columns):
        df[c]=None
    df['Source_file'] = tbl
    return(df[cols + ['Source_file']])


In [4]:
def merge_table(existing, new, key_col, field_col='FieldID'):
    # take Field IDs that are already annotated (by a user-specified column)
    annotated = set(existing[
        (existing[key_col] is not None) and 
        (existing[key_col] != '')
    ][field_col].map(lambda x: str(x)))
    return(pd.concat([
        existing, 
        new[new[field_col].map(lambda x: str(x) not in annotated)]
    ]))


In [5]:
existing_tables = [
    'ukb_20170727.tsv', 'ukb_20170818.tsv', 'ukb_20170827.tsv', 'ukb_20171015.tsv', 
    'ukb_20171110.tsv', 'ukb_20171113.tsv', 'ukb_20171211.tsv', 'ukb_20181109.tsv', 
    'ukb_20190327.tsv', 'ukb_20190406.tsv', 'ukb_20190409.tsv', 'ukb_20190412.tsv'
]


### Manual inspection of duplicated entries in the exsiting tables

In [6]:
existing_df_simple_concat = pd.concat(
    [my_read_table(x, cols) for x in existing_tables]
)
dup_entries = {k:v for k,v in collections.Counter(existing_df_simple_concat['GBE ID']).items() if v>1}


In [7]:
existing_df_simple_concat[
    existing_df_simple_concat['GBE ID'].isin(dup_entries.keys())
].sort_values(
    by=['GBE ID']
).to_csv('20191110.duplicates.tsv', sep='\t', index=False)


#### We confirmed that all of the duplicated entries are consistent (i.e. it is fine to keep an arbitrary picked one)

### Prepare existing ones with a proper merge

In [8]:
existing_df = functools.reduce(
    lambda x, y: merge_table(x, y, 'GBE ID'), 
    [my_read_table(x, cols) for x in existing_tables]
)


In [9]:
{k:v for k,v in collections.Counter(existing_df['GBE ID']).items() if v>1}


{nan: 402}

### concatenate with the new tables

In [10]:
new_tables =[
    'ukb_20191030.tsv', 
    'ukb_20191101.2004890.35059.tsv',
    'ukb_20191101.2005223.37338.tsv'
]


In [13]:
merged_df = functools.reduce(
    lambda x, y: merge_table(x, y, 'FieldID'), 
    [existing_df] + [my_read_table(x, cols) for x in new_tables]
)[cols]
merged_df['FieldID'] = merged_df['FieldID'].map(lambda x: int(x))


In [14]:
merged_df.shape

(5788, 27)

In [15]:
merged_df.sort_values(by=['FieldID', 'GBE ID'])[cols].to_csv('ukb_20191110.tsv', sep='\t', index=False)
